In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import csv
from pathlib import Path

import datetime

now = datetime.datetime.now()

In [2]:
import selenium
from selenium import webdriver

In [3]:
# konfigurasi chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

browser = webdriver.Chrome(chrome_options=chrome_options)

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [4]:

data_folder = Path("data/")
file_data = 'data_kamar_detail' + now.strftime("%Y%m%d") + '-' + now.strftime("%H%M") + '.csv'
filename = data_folder / file_data
csvFile = open(filename, 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['satker', 'nama', 'alamat', 'prov', 'ruang', 'kelas', 'total_kamar', 'terisi_lk', 'terisi_pr', 'total_terisi','kosong_lk', 'kosong_pr', 'total_kosong', 'waiting_list','last_update','lat','lng'])



148

In [5]:
faskes_df = pd.read_csv("Faskes - Rumah Sakit.csv",delimiter=',',encoding='ISO-8859-1')
faskes_df.head()

,Unnamed: 0,prov_id,kode_rs,nama_unit,alamat,nama_prov,jenis_faskes,lat,lng
0,1,12,1277015,RS TNI Tk. IV 01.07.03,Jl. Sudirman No.1 Kelurahan,SUMATERA UTARA,Rumah Sakit,3.596056,98.675812
1,2,12,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Rumah Sakit,1.419353,99.185211
2,3,12,1276040,RS Tk IV 01.07.02 Binjai,Jl. Bandung No.4 Binjai,SUMATERA UTARA,Rumah Sakit,3.699319,98.507103
3,4,12,1276025,RS Bangkatan PTPN II,Jl. Hasanuddin No.40 Binjai,SUMATERA UTARA,Rumah Sakit,3.600425,98.485710
4,5,12,1275996,RS Umum Sufina Azis,"Jl Karya Baru No.1,Medan",SUMATERA UTARA,Rumah Sakit,3.590356,98.711342


In [6]:
faskes_df.shape

(1222, 9)

In [7]:
for index, row in faskes_df.iterrows():
    satker = str(row['kode_rs'])
    nama_rs = str(row['nama_unit'])
    alamat =  str(row['alamat'])
    prov =  str(row['nama_prov'])
    lat =  row['lat']
    long =  row['lng']

    link = 'http://sirs.yankes.kemkes.go.id/integrasi/data/bed_monitor.php?satker='+satker
    browser.get(link)
    #print(r)
    data = browser.page_source
    url = soup(data,"lxml")
    

    table = url.find('table', attrs={'class':'tbl-responsive table table-striped table-bordered'})
    #print(table)

    if table is not None:
        res = []
        table_rows = table.find_all('tr')

        num_rows = len(table_rows)
        #print(satker+'-'+nama_rs)

        i = 0
        for tr in table_rows:
            _satker = satker
            _nama = nama_rs
            _alamat = alamat
            _prov = prov
            _lat = lat
            _long = long
            _ruang = '-'
            _kelas = '-'
            _total_kamar = '-'
            _terisi_lk = '-'
            _terisi_pr = '-'
            _total_terisi = '-'
            _kosong_lk = '-'
            _kosong_pr = '-'
            _total_kosong = '-'
            _waiting_list = '-'
            _last_update = '-'

            if i>1 and i<(num_rows-1):
                
                td = tr.find_all('td')
                #print(td)
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                #print(row)
                #print(str(i)+'-'+str(len(row)))

                
                if row:
                    if len(row)==12:
                        _ruang = row[1]
                        _kelas = row[2]
                        _total_kamar = row[3] 
                        _terisi_lk = row[4] 
                        _terisi_pr = row[5]
                        _total_terisi = row[6]
                        _kosong_lk = row[7]
                        _kosong_pr = row[8]
                        _total_kosong = row[9]
                        _waiting_list = row[10]
                        _last_update = row[11]
                    elif len(row)==10:
                        _ruang = '-'
                        _kelas = row[0] 
                        _total_kamar = row[1] 
                        _terisi_lk = row[2] 
                        _terisi_pr = row[3]
                        _total_terisi = row[4]
                        _kosong_lk = row[5]
                        _kosong_pr = row[6]
                        _total_kosong = row[7]
                        _waiting_list = row[8]
                        _last_update = row[9]
                    else:
                        _ruang = '-'
                        _kelas = '-'
                        _total_kamar = row[0] 
                        _terisi_lk = row[1] 
                        _terisi_pr = row[2]
                        _total_terisi = row[3]
                        _kosong_lk = row[4]
                        _kosong_pr = row[5]
                        _total_kosong = row[6]
                        _waiting_list = row[7]
                        _last_update = row[8]
                    #print(_waiting_list)
                
                    
                csvWriter.writerow([_satker, _nama, _alamat, _prov, _ruang, _kelas, _total_kamar, _terisi_lk, _terisi_pr, _total_terisi,  _kosong_lk, _kosong_pr, _total_kosong, _waiting_list, _last_update, _lat, _long])
            i = i +1


1277011-RS Umum Daerah Padang Sidempuan
1276040-RS Tk IV 01.07.02 Binjai
1275794-RS Umum Haji Medan
1275655-RSUP H. Adam Malik
1275633-RS Umum Martha Friska
1275622-RS Umum Imelda Pekerja Indonesia
1275600-RS Umum Prima Husada Cipta Medan
1275590-RS Umum Bhakti
1275276-RS Umum Vina Estetica
1275221-RS Jiwa Prof. Dr. Muhammad Ildrem
1275196-RS Umum Sari Mutiara
1275152-RS Umum Lanu Dr. Abd. Malik
1275126-RS Umum Methodist
1275046-RS Bhayangkara Medan
1275035-RS Tk. II Putri Hijau Medan
1275013-RS Umum Daerah Dr. Pirngadi
1274012-RS Umum Daerah Dr. H. Kumpulan Pane
1273102-RS Umum Horas Insani
1273055-RS Umum Vita Insani
1273011-RS Umum Daerah Dr. Djasamen Saragih
1272010-RS Umum Daerah Dr. Tengku Mansyur
1271016-RS Umum Daerah Dr. Fl Tobing Sibolga
1218125-RS Umum Daerah Sultan Sulaiman Syariful Alamsyah
1214010-RS Umum Daerah Lukas Hilisimaetano
1213013-RS Umum Daerah Tanjung Pura
1211033-RS Umum Kusta Lau Simomo
1211011-RS Umum Daerah Kabanjahe
1210010-RS Umum Daerah Sidikalang
120907

3204086-RS Umum Daerah Al Ihsan Provinsi Jawa Barat
3204041-RS Umum Kebonjati
3204016-RS Umum Daerah Majalaya
3203037-RS Umum Daerah Cimacan
3203015-RS Umum Daerah Sayang
3202051-RS Umum Daerah Jampang Kulon
3202040-RS Umum Daerah Palabuhanratu
3202014-RS Umum Daerah Sekarwangi
3201050-RS Umum Daerah Ciawi
3201046-RS Umum Daerah Cibinong
3201024-RS Paru Dr. M. Goenawan Partowidigdo
3376034-RS Umum Tk.IV Tegal
3376012-RS Umum Daerah Kardinah
3375071-RS Umum Karomah Holistic
3374353-RS Ibu dan Anak Kusuma Pradja
3374342-RS Umum Daerah K.M.R.T Wongso Negoro
3374273-RS Ibu dan Anak Anugerah
3374134-RS Umum Daerah Tugurejo Semarang
3374123-RS Jiwa Daerah Dr. Amino Gondohutomo
3374112-RS Umum Panti Wilasa Citarum
3374076-RS Umum Sultan Agung Semarang
3374043-RS Umum Telogorejo Semarang
3374021-RS Umum St. Elisabeth Semarang
3374010-RS Umum Pusat Dr. Kariadi
3373016-RS Umum Daerah Salatiga
3372234-RS Umum Daerah Kota Surakarta
3372180-RSU Islam Surakarta
3372096-RS Umum PKU Muhammadiyah
33720

5306031-RS Sito Husada
5306016-RS Umum Daerah MGR Gabriel Manek
5305015-RS Umum Daerah Kefamenanu
5304014-RS Umum Soe
5303045-RS Umum Daerah Naibonat
5303033-RS Bhayangkara
5303024-RSUD Rote Ndao Baa
5303022-RS Tk.IV Wirasakti Kupang
5302034-RS Umum Imanuel Sumba
5302023-RS Umum Daerah Waingapu
5301033-RS Umum Waikabubak
5301011-RS Kristen Lende Moripa
6172066-RS Jiwa Provinsi Kalimantan Barat
6172011-RS Umum Daerah Dr. Abdul Aziz Singkawang
6171113-RS Islam Yarsi Pontianak
6171085-RS Bersalin Jeumpa Pontianak
6171067-RS Anton Soedjarwo Pontianak
6171011-RS Umum Daerah Dr. Soedarso Pontianak
6110012-RS Umum Daerah Kabupaten Melawi
6109012-RS Umum Daerah Kabupaten Sekadau
6108011-RS Umum Daerah dr. A. Diponegoro Putussibau
6107010-RS Umum Daerah Ade Muhammad Djoen Sintang
6106020-RS Fatima Ketapang
6106016-RS Umum Daerah Dr. Agoesdjam Ketapang
6105015-RS Umum Daerah M. Th. Djaman Sanggau
6104014-RS Umum Daerah Dr. Rubini Mempawah
6103013-RS Umum Daerah Kabupaten Landak
6102012-RS Umum D

In [8]:
csvFile.close()

In [9]:
dataset = pd.read_csv(filename)
dataset.shape

(30960, 17)

In [10]:
dataset.head(100)

,satker,nama,alamat,prov,ruang,kelas,total_kamar,terisi_lk,terisi_pr,total_terisi,kosong_lk,kosong_pr,total_kosong,waiting_list,last_update,lat,lng
0,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Umum,super VIP,0,0,0,0,-,-,0,NaN,12/Feb/2019 08:45:56,1.419353,99.185211
1,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Kelas 3,6,0,0,0,-,-,6,NaN,12/Feb/2019 09:03:38,1.419353,99.185211
2,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Isolasi,4,0,0,0,-,-,4,NaN,12/Feb/2019 09:02:56,1.419353,99.185211
3,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Anak,Kelas 1,4,0,0,0,-,-,4,NaN,12/Feb/2019 09:06:30,1.419353,99.185211
4,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Kelas 2,8,0,0,0,-,-,8,NaN,12/Feb/2019 09:05:37,1.419353,99.185211
5,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Kelas 3,12,0,0,0,-,-,12,NaN,12/Feb/2019 09:04:09,1.419353,99.185211
6,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Penyakit Dalam,Kelas 2,12,0,0,0,-,-,12,NaN,12/Feb/2019 09:07:28,1.419353,99.185211
7,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Kelas 3,30,0,0,0,-,-,30,NaN,12/Feb/2019 09:06:57,1.419353,99.185211
8,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Kebidanan,Kelas 2,6,0,0,0,-,-,6,NaN,12/Feb/2019 09:08:24,1.419353,99.185211
9,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,-,Kelas 3,10,0,0,0,-,-,10,NaN,12/Feb/2019 09:07:55,1.419353,99.185211
